In [1]:
import pandas as pd

# train and test datasets
df_train = pd.read_csv("final_match_pairs_train.txt", sep = '\t') # inferred match pairs
df_test = pd.read_csv("final_match_pairs_ground_truth_additional.txt", sep = '\t') # ground-truth match pairs

# retain test dataset for testing reidentification algorithm
data_test = df_test.copy(deep = True)

# filter rows with match = 1
df_train = df_train[df_train.match == 1]
df_test = df_test[df_test.match == 1]

# drop redundant columns
index_cols = ['file', 'adv', 'stop']
df_train.drop(index_cols + ['match'], axis = 1, inplace = True)
df_test.drop(index_cols + ['match'], axis = 1, inplace = True)

# test dataset of candidate adv where travel time are to be predicted
X_adv = data_test.drop(index_cols + ['match', 'travel_time'], axis = 1)

# split training features & target into train and test sets
random_state = 42
X_train = df_train.drop('travel_time', axis = 1)
y_train = df_train.travel_time
X_test = df_test.drop('travel_time', axis = 1)
y_test = df_test.travel_time

print(f"Train dataset size: {len(X_train)}")
print(f"Test dataset size: {len(X_test)}")

Train dataset size: 5391
Test dataset size: 619


In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# PyTorch Dataset for Tabular Data
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# Differentiable Oblivious Decision Tree Layer
class DifferentiableTree(nn.Module):
    def __init__(self, input_dim, num_trees, tree_depth):
        super(DifferentiableTree, self).__init__()
        self.num_trees = num_trees
        self.tree_depth = tree_depth
        self.input_dim = input_dim

        # Linear layers for tree splits
        self.linear_layers = nn.ModuleList(
            [nn.Linear(input_dim, 2**tree_depth) for _ in range(num_trees)]
        )

        # Leaf values for each tree
        self.leaf_values = nn.ParameterList(
            [nn.Parameter(torch.randn(2**tree_depth)) for _ in range(num_trees)]
        )

    def forward(self, x):
        tree_outputs = []
        for i in range(self.num_trees):
            logits = self.linear_layers[i](x)  # Compute tree splits
            probs = torch.sigmoid(logits)  # Use sigmoid for split probabilities
            leaves = torch.matmul(probs, self.leaf_values[i])  # Weighted sum over leaves
            tree_outputs.append(leaves.unsqueeze(-1))  # Add dimension for stacking

        # Stack along the last dimension and average across trees
        return torch.cat(tree_outputs, dim=-1)  # Shape: (batch_size, num_trees)


# NODE Model
class NODEModel(nn.Module):
    def __init__(self, input_dim, num_trees=128, tree_depth=5, dropout=0.1):
        super(NODEModel, self).__init__()
        self.tree_layer = DifferentiableTree(input_dim, num_trees, tree_depth)
        self.dropout = nn.Dropout(dropout)
        self.output_layer = nn.Linear(num_trees, 1)

    def forward(self, x):
        x = self.tree_layer(x)  # Shape: (batch_size, num_trees)
        x = self.dropout(x)
        return self.output_layer(x).squeeze(-1)  # Final regression output


# Training and Validation Functions
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


def validate_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            total_loss += loss.item()
    return total_loss / len(loader)


# Perform K-Fold Cross-Validation with Hyperparameter Tuning
def k_fold_cv_NODE(X, y, param_grid, folds=5, epochs=50, patience=10):
    # Convert y to a NumPy array for indexing compatibility
    y = np.array(y)

    # Preprocessing
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # K-Fold setup
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Track best hyperparameters and score
    best_params = None
    best_loss = float('inf')
    best_model = None
    best_avg_rmse = float('inf')

    # Iterate over hyperparameter grid
    for params in tqdm(param_grid):
        fold_losses = []
        fold_rmses = []  # To store RMSE for each fold

        for train_idx, val_idx in kf.split(X_scaled):
            # Split into train and validation sets
            X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]  # No KeyError now

            # Create datasets and loaders
            train_data = TabularDataset(X_train, y_train)
            val_data = TabularDataset(X_val, y_val)
            train_loader = DataLoader(train_data, batch_size=params['batch_size'], shuffle=True)
            val_loader = DataLoader(val_data, batch_size=params['batch_size'])

            # Initialize model, optimizer, and loss
            model = NODEModel(
                input_dim=X.shape[1],
                num_trees=params['num_trees'],
                tree_depth=params['tree_depth'],
                dropout=params['dropout']
            ).to(device)

            optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
            criterion = nn.MSELoss()

            # Early stopping
            best_fold_loss = float('inf')
            no_improvement = 0

            for epoch in range(epochs):
                train_epoch(model, train_loader, criterion, optimizer, device)
                val_loss = validate_epoch(model, val_loader, criterion, device)

                if val_loss < best_fold_loss:
                    best_fold_loss = val_loss
                    no_improvement = 0
                else:
                    no_improvement += 1

                if no_improvement >= patience:
                    break

            # Store fold validation loss and RMSE
            fold_losses.append(best_fold_loss)
            fold_rmses.append(np.sqrt(best_fold_loss))  # RMSE = sqrt(MSE)

        # Calculate mean loss and RMSE across folds
        mean_loss = np.mean(fold_losses)
        mean_rmse = np.mean(fold_rmses)

        # Update best parameters and model if better performance is achieved
        if mean_loss < best_loss:
            best_loss = mean_loss
            best_avg_rmse = mean_rmse
            best_params = params
            best_model = model  # Save the best model

    print(f"Best Parameters: {best_params}, Best Avg RMSE: {best_avg_rmse}, Best Loss: {best_loss}")
    return best_model, scaler, best_params, best_avg_rmse

# Train the Final Model
def train_best_model(X, y, best_params, epochs=50, patience=10):
    # Convert y to NumPy array for compatibility with PyTorch
    y = np.array(y)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Create dataset and dataloader
    train_data = TabularDataset(X_scaled, y)
    train_loader = DataLoader(train_data, batch_size=best_params['batch_size'], shuffle=True)

    # Initialize the model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = NODEModel(
        input_dim=X.shape[1],
        num_trees=best_params['num_trees'],
        tree_depth=best_params['tree_depth'],
        dropout=best_params['dropout']
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.MSELoss()

    # Early stopping
    best_loss = float('inf')
    no_improvement = 0

    for epoch in range(epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, device)

        # No validation here since we're training on the full training set
        if train_loss < best_loss:
            best_loss = train_loss
            no_improvement = 0
        else:
            no_improvement += 1

        if no_improvement >= patience:
            print(f"Early stopping at epoch {epoch + 1}")
            break

    print(f"Final Training Loss: {best_loss}")
    return model, scaler

# Evaluate on the Test Set
def evaluate_model(model, scaler, X_test, y_test):
    # Preprocess the test data
    X_test_scaled = scaler.transform(X_test)

    # Convert to tensor and move to device
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model.eval()
    with torch.no_grad():
        X_test_tensor = X_test_tensor.to(device)
        y_pred = model(X_test_tensor).cpu().numpy()

    # Calculate MSE and RMSE
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print(f"Test RMSE: {rmse}")
    return rmse

param_grid = [
    {
        'num_trees': nt,
        'tree_depth': td,
        'learning_rate': lr,
        'batch_size': bs,
        'dropout': dp
    }
    for nt in [16, 32, 64]
    for td in [2, 3, 5]
    for lr in [0.01, 0.001]
    for bs in [32, 64]
    for dp in [0.0, 0.1, 0.2]
]

# Best Hyperparameters: {'num_trees': 32, 'tree_depth': 3, 'learning_rate': 0.01, 'batch_size': 32, 'dropout': 0.1}

# Main Function
if __name__ == "__main__":
    # Perform 5-Fold CV with Hyperparameter Tuning
    best_model, scaler, best_params, best_avg_rmse = k_fold_cv_NODE(
        X_train.values, y_train, param_grid, folds=5, epochs=50, patience=10
    )

    # Train the final model on the full training set
    final_model, final_scaler = train_best_model(
        X_train.values, y_train, best_params, epochs=50, patience=10
    )

    # Evaluate on the test set
    test_rmse = evaluate_model(final_model, final_scaler, X_test.values, y_test)

    print(f"Best Hyperparameters: {best_params}")
    print(f"Best Avg RMSE on Validation Folds: {best_avg_rmse}")
    print(f"Test RMSE: {test_rmse}")

100%|██████████████████████████████████████████████████████████████████████████████| 108/108 [4:11:29<00:00, 139.72s/it]


Best Parameters: {'num_trees': 16, 'tree_depth': 2, 'learning_rate': 0.01, 'batch_size': 32, 'dropout': 0.1}, Best Avg RMSE: 0.855632786461278, Best Loss: 0.7325745261767331
Final Training Loss: 0.7059998827925801
Test RMSE: 0.9495836936714142
Best Hyperparameters: {'num_trees': 16, 'tree_depth': 2, 'learning_rate': 0.01, 'batch_size': 32, 'dropout': 0.1}
Best Avg RMSE on Validation Folds: 0.855632786461278
Test RMSE: 0.9495836936714142


In [3]:
# Train the final model on the full training set
final_model, final_scaler = train_best_model(
    X_train.values, y_train, best_params, epochs=50, patience=10
)

# Evaluate on the test set
test_rmse = evaluate_model(final_model, final_scaler, X_test.values, y_test)

print(f"Best Hyperparameters: {best_params}")
print(f"Best Avg RMSE on Validation Folds: {best_avg_rmse}")
print(f"Test RMSE: {test_rmse}")

Final Training Loss: 0.7028120156575942
Test RMSE: 0.9396563348775868
Best Hyperparameters: {'num_trees': 16, 'tree_depth': 2, 'learning_rate': 0.01, 'batch_size': 32, 'dropout': 0.1}
Best Avg RMSE on Validation Folds: 0.855632786461278
Test RMSE: 0.9396563348775868


In [4]:
def predict_with_NODE(model, scaler, X_adv):
    # Preprocess the data using the scaler
    X_adv_scaled = scaler.transform(X_adv)

    # Convert to PyTorch tensor
    X_adv_tensor = torch.tensor(X_adv_scaled, dtype=torch.float32)

    # Move tensor to device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    X_adv_tensor = X_adv_tensor.to(device)

    # Predict
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        predictions = model(X_adv_tensor).cpu().numpy()  # Convert predictions to NumPy array

    # Return predictions as a Pandas Series
    return pd.Series(predictions, index=X_adv.index, name="predictions")

# Predict on the advanced DataFrame X_adv
predictions = predict_with_NODE(final_model, final_scaler, X_adv)

# add predicted travel time to dataset with both 1 and 0 matches
data_pred = data_test.copy(deep = True)
data_pred['y_pred'] = predictions

# save predicted travel time values
data_pred.to_csv("predicted_travel_time/NODE.txt", sep = '\t', index = False)

/home/catsit/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [5]:
import os

tt_thru_min, tt_thru_max = 2.5, 12 # min, max of through travel time to constrain search space

# function to process candidate match pairs
def reidentifyMatchPairs(adf, sdf, id_adv, data_pred, file):
    thru_match_initial = [] # store initial candidate match pairs of adv to stop-bar det
    
    for i in id_adv:
        adv_time = adf[adf.ID == i].TimeStamp.values[0]
        adv_lane = adf[adf.ID == i].Lane.values[0]

        # stop-bar det IDs on the same lane to look for a match
        id_stop_look = set(sdf[sdf.Lane == adv_lane].ID)

        for j in id_stop_look:
            stop_time = sdf[sdf.ID == j].TimeStamp.values[0]

            if stop_time > adv_time: # look forward in timestamp
                tt_adv_stop = (stop_time - adv_time) / np.timedelta64(1, 's') # paired travel time

                if tt_thru_min <= tt_adv_stop <= tt_thru_max:
                    # get predicted travel time for file and id_adv
                    Xi = data_pred.copy(deep = True)
                    Xi = Xi[(Xi.file == file[:-4]) & (Xi.adv == i)].reset_index(drop = True) # discard .txt
                    
                    tt_predict = Xi.loc[0, 'y_pred'] # predicted travel time
                    tt_diff = round(abs(tt_adv_stop - tt_predict), 4) # abs diff between paired & predicted

                    # store adv ID, stop ID, travel time diff
                    thru_match_initial.append([i, j, tt_diff])

    # dicts to store the lowest error for each adv, stop ID
    seen_adv_id, seen_stop_id = {}, {}

    # iterate through each candidate pair
    for pair in thru_match_initial:
        adv_id, stop_id, error = pair

        # check if adv ID not seen or if error is lower than seen error for that adv ID
        if (adv_id not in seen_adv_id) or (error < seen_adv_id[adv_id][1]):
            seen_adv_id[adv_id] = list([stop_id, error])

        # check if stop ID not seen or if error is lower than seen error for that stop ID
        if (stop_id not in seen_stop_id) or (error < seen_stop_id[stop_id][1]):
            seen_stop_id[stop_id] = list([adv_id, error])

    # match pairs for adv with lowest error
    df_adv = pd.DataFrame(seen_adv_id, index = ['adv', 'stop']).T.reset_index()
    df_adv.columns = ['adv', 'stop', 'error']

    # match pairs for stop with lowest error
    df_stop = pd.DataFrame(seen_stop_id, index = ['stop', 'adv']).T.reset_index()
    df_stop.columns = ['stop', 'adv', 'error']
    
    return {'df_adv': df_adv, 'df_stop': df_stop}

file_path = "data"
files = os.listdir(file_path)  # list of processed files to run through reidentifying algorithm

df_result = [] # store reidentified match pairs from each file

for file in files:
    print("Running reidentification algorithm for file: ", file)
    # read events-processed file with timestamp data
    df = pd.read_csv(os.path.join(file_path, file), sep = '\t')
    df.TimeStamp = pd.to_datetime(df.TimeStamp, format = '%Y-%m-%d %H:%M:%S.%f').sort_values()
    df.dropna(axis = 0, inplace = True) # drop rows with Nan

    # data frames for adv and stop-bar det
    adf = df[df.Det == 'adv']
    sdf = df[df.Det == 'stop']
    id_adv = list(sorted(adf.ID))

    # process candidate match pairs to get datasets of adv and stop pairs
    candidate_match_result = reidentifyMatchPairs(adf, sdf, id_adv, data_pred, file)
    df_adv = candidate_match_result['df_adv']
    df_stop = candidate_match_result['df_stop']

    # resulting common match pairs
    df_match_pair = df_adv.merge(df_stop, on = ['adv', 'stop', 'error'])
    df_match_pair['file'] = file[:-4]
    df_result.append(df_match_pair)

match_result = pd.concat(df_result)
match_result.to_csv("reidentification_result/NODE.txt", sep = '\t')

# ground-truth match pairs for index cols
match_ground = data_test.copy(deep = True)
num_candidate_pairs = match_ground.shape[0]
print(f"\nNum of candidate pairs: {num_candidate_pairs}\n")

# filter ground-truth match pairs for match == 1 and select index cols
match_ground = match_ground[match_ground.match == 1][index_cols]

# get true positive (TP), false positive (FP), and false negative (FN) matches   
match_TP = pd.merge(match_result, match_ground, on = index_cols)
match_FP = match_result.merge(match_ground, on = index_cols, how = 'left', indicator = True).query('_merge == "left_only"').drop(columns = '_merge')
match_FN = match_ground.merge(match_result, on = index_cols, how = 'left', indicator = True).query('_merge == "left_only"').drop(columns = '_merge')

# num of TP, FP, FN
TP, FP, FN = match_TP.shape[0], match_FP.shape[0], match_FN.shape[0]
TN = num_candidate_pairs - TP - FP - FN

# compute metrics
accuracy = round((TP + TN) / (TP + FP + FN + TN), 4)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2*precision*recall / (precision + recall)

print(f"TP, FP, FN: {TP}, {FP}, {FN}")
print(f"Accuracy, Precision, Recall, F1: {accuracy:.4f}, {precision:.4f}, {recall:.4f}, {f1:.4f}")

Running reidentification algorithm for file:  20230327_0700_1400.txt
Running reidentification algorithm for file:  20221206_0945_1200.txt
Running reidentification algorithm for file:  20221214_0645_0715.txt
Running reidentification algorithm for file:  20230327_1415_1900.txt
Running reidentification algorithm for file:  20221206_0845_0915.txt
Running reidentification algorithm for file:  20221214_0945_1015.txt

Num of candidate pairs: 1040

TP, FP, FN: 534, 31, 85
Accuracy, Precision, Recall, F1: 0.8885, 0.9451, 0.8627, 0.9020
